# Hoja de Trabajo 2 Deep Learning

Hoja de trabajo 2 – Deep Learning

Edwin Ortega 22305 - Esteban Zambrano 22119

Link del repositorio:<br>
https://github.com/EstebanZG999/HDT2_DL

### Task 1

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargar dataset
iris = load_iris()
X, y = iris.data, iris.target

# Normalizar
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir train/val
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Convertir a tensores
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

print("Train shape:", X_train.shape, y_train.shape)
print("Validation shape:", X_val.shape, y_val.shape)


Train shape: torch.Size([120, 4]) torch.Size([120])
Validation shape: torch.Size([30, 4]) torch.Size([30])


### Task 2

In [2]:
import torch
import torch.nn as nn

def get_activation(name: str) -> nn.Module:
    """Devuelve una activación por nombre."""
    name = name.lower()
    if name == "relu":
        return nn.ReLU()
    if name == "tanh":
        return nn.Tanh()
    if name == "leakyrelu":
        return nn.LeakyReLU()
    if name == "gelu":
        return nn.GELU()
    raise ValueError(f"Activación no soportada: {name}")

class MLPClassifier(nn.Module):
    """
    MLP simple para clasificación en Iris:
    - Arquitectura parametrizable: hidden_layers, activación, dropout.
    - Última capa produce logits (sin softmax); compat. con CrossEntropyLoss.
    """
    def __init__(
        self,
        input_dim: int,
        output_dim: int,
        hidden_layers=(16, 16),
        activation="relu",
        dropout_p=0.0,
        use_batchnorm=False,
    ):
        super().__init__()
        act = get_activation(activation)

        layers = []
        prev = input_dim
        for h in hidden_layers:
            layers.append(nn.Linear(prev, h))
            if use_batchnorm:
                layers.append(nn.BatchNorm1d(h))
            layers.append(act)
            if dropout_p and dropout_p > 0:
                layers.append(nn.Dropout(p=dropout_p))
            prev = h

        # Capa de salida (logits)
        layers.append(nn.Linear(prev, output_dim))

        self.net = nn.Sequential(*layers)

        # Inicialización razonable (Kaiming para ReLU/LeakyReLU, Xavier para otras)
        for m in self.modules():
            if isinstance(m, nn.Linear):
                if activation.lower() in ["relu", "leakyrelu"]:
                    nn.init.kaiming_uniform_(m.weight, nonlinearity="relu")
                else:
                    nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.net(x)

# Instancia recomendada para Iris
input_dim = 4   # iris.features
output_dim = 3  # 3 clases
model = MLPClassifier(
    input_dim=input_dim,
    output_dim=output_dim,
    hidden_layers=(32, 16),   
    activation="relu",        # "relu", "tanh", "gelu", "leakyrelu"
    dropout_p=0.0,
    use_batchnorm=False
)

print(model)
sum_params = sum(p.numel() for p in model.parameters())
print(f"Parámetros totales: {sum_params}")


MLPClassifier(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=3, bias=True)
  )
)
Parámetros totales: 739
